# Tutorial for fitting uranium resonances using `pleiades`
Tsviki Y. Hirsh and Alex Long

July 9th 2024

This is a notebook to guide a pleiades user to fitting a neutron transmission spectrum of uranium isotopes. For this tutorial we will be using the simulated data set `u235-u238.twenty` in `examples/data/`

##### import the needed libraries from pleiades

In [1]:
from pleiades import sammyUtils, sammyRunner, sammyOutput, sammyPlotter

### Loading a config file for SAMMY fitting

We have a uranium.ini file that contains many of the parameters that we 

In [2]:
# Load the configuration file from the ini file in the parent directory
uranium = sammyUtils.SammyFitConfig("../configFiles/uranium.ini")

#### <span style="color:orange">What happend:</span> 
Pleiades has now created a couple of directories base on the keys and vlaues under ['directories'] in makeCompoundFit_Eu.in

Given that sammy needs to use ascii files to run, and also generates additional files that need to be parsed. We have to create a directory that contains all the needed subfolders to configure, execute, and interpret sammy runs. This is done by creating a archive_dir, which can be set in the makeCompoundFit_Eu.ini config file. If none is given then it defualts to ".archive"

Currently nothing is stored in these directories, but we will start filling them with sammy files. i

In [ ]:
!ls -lah 
!ls -lah analysis

#### Now Create the needed parFiles from ENDF for the Eu isotopes

Here we are using the create_parFile_from_endf() funtion with the natEu SammyFitConfig object to create the needed parFiles for each of the Eu isotopes list in the makeCompoundFit_Eu.ini file. 

Note: You can change the print verbose level to print out important variables and paths. 

In [ ]:
# Create the needed parFiles from ENDF for the isotopes in the configuration file
sammyUtils.create_parFile_from_endf(uranium, verbose_level=1)

#### <span style="color:orange">What happend:</span> 

Pleiades configured two SAMMY runs to generate the needed resonance parameters for the final SAMMY fit. These ENDF SAMMY runs are stored in the 'endf_dir' which is set in makeCompoundFit_Eu.ini

If you `ls` the .archive/endf directory, you will see two sub-dirs: Eu151 & Eu153. Each of these folders contains the results for running SAMMY to generate SAMNDF.PAR files for each respective isotope.

In [ ]:
!ls -lah analysis/endf/
!ls -lah analysis/endf/U-235/

#### Configure SAMMY for the fit of Uranium foil

Now that we have the needed SAMNDF.PAR files for each isotope (stored in the .archive/endf/Eu15* folders) we can configure all the needed SAMMY files to fit a naterual Eu transmission spectrum. 

The data for this is stored in `examples/data` and the path is set in makeCompoundFit_Eu.ini as `data_dir = "../data"`

In [ ]:
# Configure the sammy run, this will create a compound parFile.
sammy_run = sammyUtils.configure_sammy_run(uranium, verbose_level=1)

#### <span style="color:orange">What happend:</span> 

Here Pleiades has taken all the needed information from the two (Eu151, Eu153) `SAMNDF.PAR` files and combined them to form a `params.par` file. Additionally Pleiades has created the needed input.inp file to run the natEu SAMMY fit.  

In [ ]:
!ls -lah analysis/u235-u238/

#### Running the SAMMY fit on the Uranium foil data

Now that everything is set we are prepared to run the actual fit on the natural Eu data. 
Note: You can change the print verbose level to print out important variables and paths. 

In [ ]:
# Run a sammy fit.
sammyRunner.run_sammy_fit(sammy_run, verbose_level=1)

#### <span style="color:orange">What happend:</span> 

Here Pleiades should have successfully ran a SAMMY fit on the simulated Urainum transmission data. If is was not successful then we should have gotten an print statement telling us the SAMMY fit failed. 

#### Parsing the output of the SAMMY run on uranium foil. 

Now we can use Pleiades to analyze the results of the SAMMY fit using the `sammyOutput` library

In [ ]:
# Grab the results from the SAMMY fit and plot the fit
results_lpt = uranium.params["directories"]["sammy_fit_dir"] / "results/SAMMY.LPT"
uranium_fit = sammyOutput.lptResults(results_lpt)

uranium_fit._print_iteration(-1)

Now we can plot the intial and final results, along with the data to see how well SAMMY was able to fit the data. 

In [ ]:
# Plot the results
results_lst = uranium.params["directories"]["sammy_fit_dir"] / "results/SAMMY.LST"
sammyPlotter.process_and_plot_lst_file(
    results_lst, residual=True, quantity="transmission"
)

#### Something is missing!

Looking at the plot it appears that we are not capturing all the isotopes! We should think about what isotopes we should add to get a better fit. 

* First list the isotopes and abundances, 
* Then we can update the abundances based on the lptresults
* Finally we can add another isotope, like Ta (hint: its Ta)

Additionally, we can change the name of the `sammy_fit_dir` in case we want to compare new fit results to the original. 

In [ ]:
# Print out the initial parameters for the isotopes (names and abundances)
print(f"Isotopes: {uranium.params['isotopes']['names']}")
print(f"Initial Abundance: {uranium.params['isotopes']['abundances']}")
print(f"Initial Thickness: {uranium.params['broadening']['thickness']}")

# Update the isotope abundances with the new results from the SAMMY fit
for i, isotope in enumerate(uranium.params["isotopes"]["names"]):
    uranium.params["isotopes"]["abundances"][i] = float(
        uranium_fit._results["Iteration Results"][-1]["Nuclides"][i]["Abundance"]
    )

# Update the sample thickness with the new results from the SAMMY fit
uranium.params["broadening"]["thickness"] = float(
    uranium_fit._results["Iteration Results"][-1]["Thickness"]
)

# check your work!
print(f"New Abundance: {uranium.params['isotopes']['abundances']}")
print(f"New Thickness: {uranium.params['broadening']['thickness']}")

In [ ]:
# Create new fit directory inside the working directory with the added isotope and its initial abundance
uranium.params["directories"]["sammy_fit_dir"] = (
    uranium.params["directories"]["working_dir"] / "u235-u238-ta181"
)

# check your work!
print(f"New fit directory: {uranium.params['directories']['sammy_fit_dir']}")

# Add Ta to the isotopes names and set initial abundance to 0.01
uranium.params["isotopes"]["names"].append("Ta-181")
uranium.params["isotopes"]["abundances"].append(0.01)

# check your work!
print(f"Names: {uranium.params['isotopes']['names']}")
print(f"Abundance: {uranium.params['isotopes']['abundances']}")

### Create a SAMMY parameter file from ENDF for the newly added Ta-181 isotope.  

Now that we have added the third isotope of Ta-181, we need to have Pleiades pull and create a parFile that we can pull resonance parameters from 

In [ ]:
# Create a new par file for the additional isotope Ta-181
if uranium.params["run_endf_for_par"]:
    sammyUtils.create_parFile_from_endf(uranium, verbose_level=1)

#### Check the archive file to see if a Ta181 sub-folder was created> 

In [ ]:
!ls -lah analysis/endf/

### Configure SAMMY inputs and parameter files with new set of isotopes. 

Now that we have added the third isotope of Ta-181, we need to have Pleiades reconfigure the needed SAMMY files to execute a SAMMY fit using all three isotopes.

In [ ]:
# Configure the sammy run, this will create a compound parFile.
sammy_run = sammyUtils.configure_sammy_run(uranium, verbose_level=1)

#### `ls` the directory to see if anything was updated. 

In [ ]:
!ls -lah analysis/u235-u238-ta181/

### Execute a SAMMY fit with the updated inputs and parameter files 

Now that everything is set, we can run SAMMY with the updated input and parameter files (keeping the same data file).

Again, we want to make sure that we get the "SAMMY executed successfully." print out. Otherwise we need to check output.out


In [ ]:
# Run the new sammy fit.
sammyRunner.run_sammy_fit(sammy_run, verbose_level=1)

#### Grab the results of the SAMMY fit using the `lptResults` class 

In [ ]:
# Grab the results from the SAMMY fit and plot the fit
updated_results_lpt = (
    uranium.params["directories"]["sammy_fit_dir"] / "results/SAMMY.LPT"
)
updated_uranium_fit = sammyOutput.lptResults(updated_results_lpt)

updated_uranium_fit._print_iteration(-1)

#### <span style="color:red">Bug:</span> Pleiades is only printing two isotopes. 

#### Plot the final results!

In [ ]:
# Plot the results
updated_results_lst = (
    uranium.params["directories"]["sammy_fit_dir"] / "results/SAMMY.LST"
)
sammyPlotter.process_and_plot_lst_file(
    updated_results_lst, residual=True, quantity="transmission"
)